In [1]:
import kwikfile as kf
import numpy as np
import imp
%matplotlib inline
import matplotlib.pyplot as plt
from klusta.kwik import KwikModel
import matplotlib as mpl
from pylab import *

import ipywidgets as widgets
from IPython.display import display
import time

class HeatMap:
    
    def __init__(self,kpath=None,name=None):
        if kpath is not None:
            self.kwik_model=KwikModel(kpath)
            self.kwikfile=kf.KwikFile(kpath)
            self.kpath=kpath
            self.name=self.kwikfile.name
            #print(self.kwikfile.list_of_groups())
    #média das amplitudes de cada site
    def shank_amplitude (self,group_name=None):
        
        
        if group_name is not None:
            if group_name is "mua" or group_name is "good" or group_name is "noise" or group_name is 'unsorted':
                f = widgets.IntProgress(min=0, max=len(self.kwikfile.clusters(group_name=group_name)))
                display(f)
                median_amplitude_group = np.zeros((10,32))
                #median_amplitude_group=np.zeros((32,32)) #!!!!!
                #print(group_name)
                clusters = self.kwik_model.spike_clusters
                for cluster_id in self.kwikfile.clusters(group_name=group_name):
                    #print(cluster_id)
                    
                    idx=np.argwhere (clusters==cluster_id)
                    w=self.kwik_model.all_waveforms[idx]
                    num_channels = w.shape[2]
                    #print(cluster_id)
                    #median_amplitude_group=np.zeros((w.shape[2],w.shape[3]))
                    #median_amplitude_group = np.mean(w[:,:,:],0)
                    
                    for ch in range (0,num_channels):
                        #print(ch)
                        #mu_spikes = np.mean(w[:,:,ch],0)
                        median_amplitude_group[ch,:]+=np.mean(w[:,:,ch],0)
                    f.value += 1
                #print(w[:,:,:])
                return median_amplitude_group
            else:
                print("Enter the group name valid!")
                return None
          
    #contorno
    '''
    def probe_geometry (self,num_channels=None):
        if num_channels is not None:
            eletrode_form = [11,6,1,0,0,0,1,6,11]
            return eletrode_form
        else:
            raise NameError("Probe geometry invalid!")
    '''
    #
    def amplitudes_heat_map (self, median_amplitude_group=None):
        fig,ax = plt.subplots(figsize=(6, 10))
        
        #print(median_amplitude_group)
        max_amplitude=np.zeros((len(median_amplitude_group[:,0])))
        #máximo de cada canal
        for ch in range(0,len(median_amplitude_group[:,0])):
            top=max(median_amplitude_group[ch,:])
            low=min(median_amplitude_group[ch,:])
            if top>abs(low):
                max_amplitude[ch]=top
            else:
                max_amplitude[ch]=abs(low)
        #
        posx=np.flipud (self.kwik_model.channel_positions [:,0])
        posy=np.flipud (self.kwik_model.channel_positions [:,1])
        print(posx,posy)
        #mapa de cores
        cmap = mpl.cm.jet
        norm = mpl.colors.Normalize(vmin=min(max_amplitude), vmax=max(max_amplitude))
        cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                norm=norm,
                                extend='both',
                                extendfrac='auto',
                                spacing='uniform',
                                orientation='vertical')  
        
        #lista de posições dos círculos
        a=list()
        dx = 35
        for ch in range (0,len(median_amplitude_group[:,0])):
            if ch%2!=0:
                x_offset = posx[ch]+dx
            else:
                x_offset = posx[ch]-dx
            y_offset =posy[ch]*2
            circle = plt.Circle((x_offset,y_offset),10,color=cb1.to_rgba(max_amplitude[ch]))
            a.append(circle)
            
        #ax = plt.gca()
        ax.cla()   
        ax.set_xlim((min(posx)-dx-10, max(posx)+dx+10))
        ax.set_ylim((min(posy)*2-10, max(posy)*2+10))
        
        for i in range(0,len(median_amplitude_group[:,0])):
            c=a[i]
            ax.add_artist(c)
            
        data = np.clip(randn(0, 0), -1, 1)
        cax = ax.imshow(data, interpolation='None', cmap=cm.jet)
        cbar = fig.colorbar(cax,ticks=[0,0.5,1],orientation='vertical')
        cbar.ax.set_yticklabels(['Low','Medium','High'])
        plt.axis('off')
        
        plt.show()

In [2]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Text
import ipywidgets as widgets

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='center'
)

data = [
    Box([Label(value='Path'),
            Text("/home/harison/UFPE/data/g01.kwik")
            ], layout=form_item_layout),
    Box([Label(value='Group name:'),
         Dropdown(options=['good', 'mua', 'noise'])],layout=form_item_layout)
    
]

form = Box(data, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))
display(form)

button = widgets.Button(description="Enviar informações!")
display(button)

def on_button_clicked(b):
    #kpath=form_items[0].children[0].value
    kpath=data[0].children[1].value
    group_name=data[1].children[1].value
    hm = HeatMap(kpath)
    ampl=hm.shank_amplitude(group_name)
    hm.amplitudes_heat_map(ampl)
    #group_name=
    #print("%s: %s" %(children[0].description,children[0].value))
    
    
button.on_click(on_button_clicked)

Box(children=(Box(children=(Label(value='Path'), Text(value='/home/harison/UFPE/data/g01.kwik')), layout=Layout(display='flex', flex_flow='row', justify_content='center')), Box(children=(Label(value='Group name:'), Dropdown(options=('good', 'mua', 'noise'), value='good')), layout=Layout(display='flex', flex_flow='row', justify_content='center'))), layout=Layout(align_items='stretch', border='solid 2px', display='flex', flex_flow='column', width='50%'))

Button(description='Enviar informações!', style=ButtonStyle())

In [13]:


a=widgets.IntProgress(
    value=2,
    min=0,
    max=10,
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)
display(a)

IntProgress(value=2, description='Loading:', max=10)

In [ ]:
import ipywidgets as widgets
from IPython.display import display	
import time

int i
w=widgets.FloatProgress(
    value=1.0,
    min=0,
    max=10.0,
    step=0.1,
    description='Loading:',
    orientation='horizontal'
)

display(w)

#while True:
while True:
    
    w.value = 1


In [36]:
import ipywidgets as widgets
from IPython.display import display	
import time

f = widgets.IntProgress(min=0, max=100)
display(f)
for i in range(0,100):
    f.value += 1

IntProgress(value=0)